In [1]:
import pandas as pd
import re
import time
from binance.client import Client
from datetime import datetime, timedelta
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta

In [2]:
ultima_cotizacion = []
ultima_fecha_cotizacion = []
prediccion = []
fecha_prediccion = []

In [3]:
def prediccion_btc_test():

   # Inserta tus credenciales de API de Binance
   api_key = 'tu_api_key'
   api_secret = 'tu_api_secreta'

   # Inicializa el cliente de Binance
   client = Client(api_key, api_secret)

   # Calcula la fecha y hora de inicio (24 horas atrás desde ahora)
   start_time = datetime.now() - timedelta(days=1)

   # Convierte el objeto datetime a un timestamp en milisegundos
   start_timestamp = int(start_time.timestamp() * 1000)

   # Obtiene los datos de cotización de Bitcoin para las últimas 24 horas con una frecuencia de 1 minuto
   historical_data = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, start_str=str(start_time))

   # Extrae los precios de cierre y el volumen operado de los datos históricos
   timestamps = [data[0] for data in historical_data]
   prices = [float(data[4]) for data in historical_data]
   volumes = [float(data[5]) for data in historical_data]

   # Verifica si las listas de precios de cierre y volumen operado tienen la misma longitud
   if len(prices) != len(volumes):
       raise ValueError("Las listas de precios de cierre y volumen operado no tienen la misma longitud")

   # Crea un DataFrame con los datos de precios de cierre y volumen operado
   df = pd.DataFrame({"Timestamp": timestamps, "Close": prices, "Volume": volumes})

   # Convierte la marca de tiempo a formato datetime y establece como índice
   df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')

   # Restar 3 horas a la columna de marca de tiempo
   df['Timestamp'] -= timedelta(hours=3)

   df.set_index('Timestamp', inplace=True)      


   # Modelo:
   #--------------------------------------------------------------------------------------------

   # Preprocesamiento de datos
   prices = df['Close'].values.reshape(-1, 1)  # Obtener los precios de cierre como matriz
   scaler = MinMaxScaler(feature_range=(0, 1))  # Escalar los precios entre 0 y 1
   scaled_prices = scaler.fit_transform(prices)  # Escalar los precios

   # Función para crear secuencias de datos
   def create_sequences(data, seq_length):
       X, y = [], []
       for i in range(len(data) - seq_length):
           X.append(data[i:(i + seq_length), 0])
           y.append(data[i + seq_length, 0])
       return np.array(X), np.array(y)

   # Longitud de la secuencia de entrada (ajustable según sea necesario)
   seq_length = 60  # Esto representaría una hora de datos con intervalos de 5 minutos

   # Crear secuencias para el conjunto de datos completo
   X_data, y_data = create_sequences(scaled_prices, seq_length)

   # Reshape de los datos para que sean compatibles con la entrada de la RNN (número de muestras, número de pasos de tiempo, número de características)
   X_data = np.reshape(X_data, (X_data.shape[0], X_data.shape[1], 1))

   # Construir modelo de RNN
   model = Sequential()
   model.add(LSTM(units=50, return_sequences=True, input_shape=(X_data.shape[1], 1)))
   model.add(LSTM(units=50))
   model.add(Dense(units=1))

   # Compilar modelo
   model.compile(optimizer='adam', loss='mean_squared_error')

   # Entrenar modelo con todo el conjunto de datos
   model.fit(X_data, y_data, epochs=100, batch_size=32)

   # Hacer predicciones para los próximos 10 minutos
   # Tomar los últimos 50 datos de prueba para predecir los próximos 10 minutos
   last_data = scaled_prices[-seq_length:].reshape(1, seq_length, 1)
   next_10_minutes_prediction = model.predict(last_data)

   # Invertir la escala de la predicción para obtener el valor real
   next_10_minutes_prediction = scaler.inverse_transform(next_10_minutes_prediction)

   # Obtener la fecha y hora para la predicción
   last_timestamp = df.index[-1]  # Último timestamp en el conjunto de datos
   next_timestamp = last_timestamp + timedelta(minutes=10)


   # Creacion DF:
   #--------------------------------------------------------------------------------------------
   

   #Creacion de listas con variables para el DF: 
   ultima_cotizacion.append(int(df['Close'].iloc[-1]))
   ultima_fecha_cotizacion.append(df.index[-1])
   prediccion.append(int(next_10_minutes_prediction))
   fecha_prediccion.append(next_timestamp)

   #Creacion del DF:
   df_final = pd.DataFrame ()
   df_final['ultima_cotizacion'] = ultima_cotizacion
   df_final['ultima_fecha_cotizacion'] = ultima_fecha_cotizacion
   df_final['prediccion'] = prediccion
   df_final['fecha_prediccion'] = fecha_prediccion

   return(df_final)

In [4]:
while True:
    try:
        df = prediccion_btc_test()
        print(df)
    except Exception as e:
        print("Error al cargar los datos:", e)
    time.sleep(450)

c:\Users\Emi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - loss: 0.1008
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0019
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0011
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0011
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0011
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 9.6565e-04
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0011
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0010
Epoch 9/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 8.9448e-04
Epoch 10/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0010
Epoch 11/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 8.6021e-04
Epoch 12/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 9.1085e-04
Epoch 13/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 8.1528e-04
Epoch 14/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 7.9210e-04
Epoch 15/100
49/49 ━━━━━━━━━━━━━━

C:\Users\Emi\AppData\Local\Temp\ipykernel_12080\2897825038.py:97: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediccion.append(int(next_10_minutes_prediction))


Epoch 1/100


c:\Users\Emi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

C:\Users\Emi\AppData\Local\Temp\ipykernel_12080\2897825038.py:97: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediccion.append(int(next_10_minutes_prediction))


Epoch 1/100


c:\Users\Emi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 0.0551
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0018
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0014
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0014
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0012
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0012
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0012
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0012
Epoch 9/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0012
Epoch 10/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0012
Epoch 11/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0011
Epoch 12/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0011
Epoch 13/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0011
Epoch 14/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0011
Epoch 15/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0011
E

C:\Users\Emi\AppData\Local\Temp\ipykernel_12080\2897825038.py:97: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediccion.append(int(next_10_minutes_prediction))
